<a href="https://colab.research.google.com/github/theplaylabs/learning/blob/main/recursive_reprompting_and_revision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: https://arxiv.org/pdf/2210.06774.pdf

Get your openai api key from settings: https://platform.openai.com/account/api-keys

In [1]:
%pip install openai --upgrade


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\mattj\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/9b/60/d3925bbd89c19dfa00eefc91ea5109b6cb2a9b1c133a2519c28277738ed1/openai-1.54.0-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for jiter<1,>=0.4.0 from https://files.pythonhosted.org/packages/96/19/e9b32c69c6dea404d983847e92cf86c3287b0f2f3e7621180a544c0ff153/jiter-0.7.0-cp311-none-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions<5,>=4.11 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/389.3 kB ? eta -:--:--
   --- ----------------------------------- 30.7/389.3 kB 640.0 kB/s eta 0:00:01
 

In [ ]:
# Get the openai secret key
import getpass
from openai import OpenAI

secret_key = getpass.getpass("Please enter your openai key: ")
import os
os.environ["OPENAI_API_KEY"]= secret_key


In [ ]:

# OpenAI set up:
client = OpenAI()
client.api_key = secret_key
# Set up OpenAI client with the provided API key

def complete(prompt, stop=None):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
            "role": "user",
            "content": prompt
            }
        ],
        stop=stop
    )
    return response.choices[0].message.content

complete("is this working?")

'Yes, it is working. How can I assist you today?\n'

In [9]:
# Add your premise:
premise_input = "A 40 year old couple from Canada move to Mexico City in search of a more fun, interesting life." #@param {type:"string"}
premise = f"Premise: {premise_input}"
premise

'Premise: A 40 year old couple from Canada move to Mexico City in search of a more fun, interesting life.'

## Plan Module

In [10]:
# Setting:
setting_prompt = f"{premise}\nThe story is set in"
setting_response = complete(setting_prompt, stop=["."])
setting = f"Setting: The story is set in {setting_response}."
setting

'Setting: The story is set in the bustling, vibrant streets of Mexico City.'

In [14]:
# Characters
num_characters = 2
characters = ""
character_format = "Please invent characters for this story, in the format of [name] is [context]. For example 'Liza Turner is a 22-year-old woman' or 'Peyton Turner is Liza's older sister'. Each character is distinct and does not reuse the same name as these examples."
character_prompt = f"Premise: {premise}\n\nSetting: {setting}\n\n{character_format}\n\n"
for n in range(num_characters):
    character_prompt += f"{n+1}. Character Portrait:\n"
    character_response = complete(f"{character_prompt}\n\n{n+1}. Character Portrait:\n", stop=["."])
    character_prompt += f"{character_response}\n\n"
    characters += f"{n+1}. Character Portrait:\n{character_response.strip()}.\n\n"

print(characters)

1. Character Portrait:
Thomas Hawthorne is a 40-year-old Canadian man known for his quick wit and adventure-loving spirit.

2. Character Portrait:
Emma Hawthorne is Thomas' cheerful, extroverted wife who sparks conversations effortlessly.




In [15]:
# Plot
plot_prompt = f"{premise}\n\n{setting}\n\n{characters}Outline the main plot points of the story.\n1."
plot_response = complete(plot_prompt)
plot = f"{plot_response}"
print("Outline the main plot points of the story\n\n1. ", plot)

Outline the main plot points of the story

1.  Despite living comfortable lives in Canada, the Hawthornes feel unfulfilled and yearn for more excitement and adventure. After much consideration, they decide to make a bold move - they sell their house and belongings and move to Mexico City.
 
2. Upon their arrival, the Hawthornes are immediately captivated by the vibrant culture and lively atmosphere of the city. The echoes of Mexican music in the streets, the colorful architecture, and the savory smell of local food in the air - all of it is exciting and new. 

3. The Hawthornes find a beautiful, colonial-style apartment in the heart of the city. They begin assimilating into their new community by learning Spanish and taking part in local customs and festivals.

4. Emma, with her extroverted personality, quickly becomes friends with her neighbors and local vendors. She fills their home with Mexican artifacts, grows to love spicy food and even starts a blog to document their adventures.


In [16]:
# Full Plan Module
print(f"{premise}\n\n{setting}\n\n{characters}\n\nOutline the main plot points of the story\n\n1. {plot}")

Premise: A 40 year old couple from Canada move to Mexico City in search of a more fun, interesting life.

Setting: The story is set in the bustling, vibrant streets of Mexico City.

1. Character Portrait:
Thomas Hawthorne is a 40-year-old Canadian man known for his quick wit and adventure-loving spirit.

2. Character Portrait:
Emma Hawthorne is Thomas' cheerful, extroverted wife who sparks conversations effortlessly.



Outline the main plot points of the story

1. Despite living comfortable lives in Canada, the Hawthornes feel unfulfilled and yearn for more excitement and adventure. After much consideration, they decide to make a bold move - they sell their house and belongings and move to Mexico City.
 
2. Upon their arrival, the Hawthornes are immediately captivated by the vibrant culture and lively atmosphere of the city. The echoes of Mexican music in the streets, the colorful architecture, and the savory smell of local food in the air - all of it is exciting and new. 

3. The Haw

## Draft Module

In [17]:
import re

plot_points_raw = plot.split('\n')
plot_points = []

for pp in plot_points_raw:
    if pp == '':
        continue
    else:
        stripped_pp = re.sub(r'^\d+\.\s*', '', pp)
        plot_points.append(stripped_pp)

print(plot_points)

['Despite living comfortable lives in Canada, the Hawthornes feel unfulfilled and yearn for more excitement and adventure. After much consideration, they decide to make a bold move - they sell their house and belongings and move to Mexico City.', ' ', 'Upon their arrival, the Hawthornes are immediately captivated by the vibrant culture and lively atmosphere of the city. The echoes of Mexican music in the streets, the colorful architecture, and the savory smell of local food in the air - all of it is exciting and new. ', 'The Hawthornes find a beautiful, colonial-style apartment in the heart of the city. They begin assimilating into their new community by learning Spanish and taking part in local customs and festivals.', 'Emma, with her extroverted personality, quickly becomes friends with her neighbors and local vendors. She fills their home with Mexican artifacts, grows to love spicy food and even starts a blog to document their adventures.', 'Thomas, an adventure enthusiast, embarks 

In [18]:
drafts = []

for idx in range(len(plot_points)):
    if idx == 0:
        previous = ""
        immediately = ""
    else:
        previous = f"Previous story summary:\n{plot_points[idx-1]}"
        immediately = f"Immediately before the current passage:\n{drafts[idx-1]}"

    upcoming = f"In the upcoming passage,\n{plot_points[idx]}"

    draft_prompt = f"Relevant context:\n{characters}{previous}\n\n{immediately}\n\n{upcoming}\n\nFull text below:"

    draft_response = complete(draft_prompt)
    draft_response = draft_response.strip()

    print(f"Plot Point {idx+1}\n-----\n", draft_prompt, f"\n{draft_response}\n-----\n\n")
    drafts.append(draft_response)

Plot Point 1
-----
 Relevant context:
1. Character Portrait:
Thomas Hawthorne is a 40-year-old Canadian man known for his quick wit and adventure-loving spirit.

2. Character Portrait:
Emma Hawthorne is Thomas' cheerful, extroverted wife who sparks conversations effortlessly.





In the upcoming passage,
Despite living comfortable lives in Canada, the Hawthornes feel unfulfilled and yearn for more excitement and adventure. After much consideration, they decide to make a bold move - they sell their house and belongings and move to Mexico City.

Full text below: 
There was a restlessness that simmered within Thomas Hawthorne, a 40-year-old Canadian man who was full of vivacity, quick wit, and an unquenchable thirst for adventure. His cheerful wife, Emma Hawthorne, seemed to share the same sentiments. With her easy sociability and outgoing personality, she could strike up conversations with anyone, anywhere. This lively pair, despite enjoying a comfortable life in Canada, often found the

In [19]:
print("".join(drafts))

There was a restlessness that simmered within Thomas Hawthorne, a 40-year-old Canadian man who was full of vivacity, quick wit, and an unquenchable thirst for adventure. His cheerful wife, Emma Hawthorne, seemed to share the same sentiments. With her easy sociability and outgoing personality, she could strike up conversations with anyone, anywhere. This lively pair, despite enjoying a comfortable life in Canada, often found themselves longing for something more exciting, something more fulfilling.

One evening, as the crimson sun was sinking into the horizon and dappled lights filled their cosy living room, the Hawthornes found themselves engaged in a deep, contemplative conversation.

"Do you ever feel like we're missing out, Thomas?" Emma glanced at her husband, hazel eyes pensive yet purposeful. "There's so much of the world we haven't seen, so many experiences we haven't had."

"I have been thinking the same," Thomas replied, a thoughtful frown furrowing his brow. "What if we did s

## Revision

In [20]:
# Progressive extraction
facts = []
edits = []
for idx in range(1, len(drafts)):
    # Old Facts
    facts_prompt = f"What facts can be inferred from this text?\n\nText:\n{drafts[idx-1]}\n\nFacts:\n-"
    facts_response = complete(facts_prompt)
    facts_response = facts_response.strip()
    facts.append(facts_response)

    # New Facts
    facts_prompt = f"What facts can be inferred from this text?\n\nText:\n{drafts[idx]}\n\nFacts:\n-"
    facts_response = complete(facts_prompt)
    facts_response = facts_response.strip()

    # Consistency
    joined_facts = "\n".join(facts)
    consistency_prompt = f"Are any New Facts inconsistent with Old Facts?:\n\Old facts:\n{joined_facts}\n\nNew facts:{facts_response}\n\nInconsistencies:\n-"
    consistency_response = complete(consistency_prompt)

    # Rewriting
    edit_prompt = f"Edit so that any inconsistencies are addressed:\nInconsistencies:{consistency_response}\n\nText:{drafts[idx]}\n\nRewritten Text:"
    edit_response = complete(edit_prompt)
    edit_response = edit_response.strip()
    edits.append(edit_response)

    print(f"Draft Text {idx+1}\n-----\n", drafts[idx], f"\n\nOld Facts:\n{joined_facts}", f"\n\nNew Facts:\n{facts_response}", f"\n\nConsistency:\n{consistency_response}", f"\n\nRewritten Text:\n{edit_response}\n-----\n\n")


Draft Text 2
-----
 Adventure waited for them at every corner of their new life. They were foreigners in a beautiful new land, each alleyway a potential story, each sunset an enchanting painting - every day, a novel experience. Mexico City was a breath of fresh air for the Hawthornes, a city that, much like themselves, thrived on excitement.

Their new home was a quaint, colourful villa tucked away in one of the city’s vibrant neighborhoods - a blooming wonderland of beautiful murals, bustling food markets, and architectural marvels. They fast became regulars at the local taqueria, their palates tickled by the burst of flavours from spicy carnitas, tender al pastor and sensational street food on every corner.

Thomas and Emma also spent their days navigating the city's historic precincts, where they marveled at grand cathedrals, ancient Aztec ruins, and museums rich with cultural antiquities. They were living the adventure they'd always longed for - soaking in tradition, exploring unch

In [21]:
"/n".join(edits)

"Adventure beckoned at every turn in their new life. The Hawthornes, foreigners in a resplendent new land, identified each alley as a potential narrative, each sunset as a captivating artwork, making every day a novel experience. For them, Mexico City was a breath of fresh air, a city that thrived on excitement, just like they did.\n\nTheir new dwelling was a charming, colourful villa nestled in one of the city’s bustling neighbourhoods - a thriving paradise of striking murals, busy food markets, and architectural wonders. Before long, they were regulars at the local taqueria, their taste buds delighted by the explosion of flavours from spicy carnitas, tender al pastor, and the sensational street food found around every corner.\n\nThomas and Emma filled their days with explorations of the city's historical districts, marveling at majestic cathedrals, ancient Aztec ruins, and museums overflowing with cultural relics. They were living the adventure they had always craved, immersing thems

In [22]:
print(edits)

["Adventure beckoned at every turn in their new life. The Hawthornes, foreigners in a resplendent new land, identified each alley as a potential narrative, each sunset as a captivating artwork, making every day a novel experience. For them, Mexico City was a breath of fresh air, a city that thrived on excitement, just like they did.\n\nTheir new dwelling was a charming, colourful villa nestled in one of the city’s bustling neighbourhoods - a thriving paradise of striking murals, busy food markets, and architectural wonders. Before long, they were regulars at the local taqueria, their taste buds delighted by the explosion of flavours from spicy carnitas, tender al pastor, and the sensational street food found around every corner.\n\nThomas and Emma filled their days with explorations of the city's historical districts, marveling at majestic cathedrals, ancient Aztec ruins, and museums overflowing with cultural relics. They were living the adventure they had always craved, immersing them

In [24]:
# Specify the filename
filename = 'output.txt'

# Open the file in write mode
with open(filename, 'w') as file:
    # Iterate through the list and write each string to the file
    for line in edits:
        file.write(line + '\n')  # Add a newline character after each string

print(f"The content has been successfully written to {filename}.")

The content has been successfully written to output.txt.
